# Prediction and Evaluation with Gaussian Process regression (GPR) model

In this exercise we will train a Gaussian Process regression (GPR) model on a training dataset and use it to make predictions on a test dataset. We will then evaluate the model by comparing the predicted values to the true values and by computing the [mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error) (MSE) and [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) ($R^2$) error metrics.

## Dependencies

First we import the dependencies.

If you are in Colab, you need to install the [GPyTorch](https://gpytorch.ai/) package by uncommenting and running the line `!pip3 install gpytorch` below before proceeding.

In [ ]:
# install dependencies
# !pip3 install gpytorch


In [ ]:
from matplotlib import pyplot as plt
import torch
import gpytorch


## Data

First we will define some data. In this and the remianing exercises we will consider synthetic data generated by the Schwefel function. We will use two input dimensions as this allows us to easily visualize the data and the predictions.

We first visualize the function on a grid of input points and then we sample a training dataset with a small amount of additive observation noise. 

In [ ]:
def schwefel(x):
    """The Schwefel function has many local optima."""
    return 418.9829 * x.shape[-1] - (x * torch.sin(torch.sqrt(torch.abs(x)))).sum(dim=-1)

def noisy_schwefel(x, noise_std=1.0):
    """The Schwefel function with observation noise."""
    return schwefel(x) + noise_std * torch.randn(x.shape[0])

def standardize(y):
    """Standardize a vector to have zero mean and unit standard deviation."""
    return (y - y.mean()) / y.std()

# Define a grid of points on which to evaluate the function
n_grid = 100
levels = 30
x_min = torch.tensor([0, 0])
x_max = torch.tensor([430, 430])

x0 = torch.linspace(0, 1, n_grid)
x1 = torch.linspace(0, 1, n_grid)
g0, g1 = torch.meshgrid(x0, x1, indexing="xy")
x_grid = torch.stack((g0.reshape(-1), g1.reshape(-1)), 1)

y_grid = schwefel(x_grid * (x_max - x_min) + x_min)
y_grid = standardize(y_grid)

vmin, vmax = y_grid.min(), y_grid.max()

plt.figure(figsize=(5,4))
plt.title("Schwefel function")
plt.contourf(x0.numpy(), x1.numpy(), y_grid.reshape(n_grid, n_grid).numpy(), vmin=vmin, vmax=vmax, levels=levels)
plt.colorbar()
plt.show()


In [ ]:
# Sample a training set and plot it
n_train = 50

torch.manual_seed(0)
x_train = torch.rand(n_train, 2)
y_train = noisy_schwefel(x_train * (x_max - x_min) + x_min)
y_train = standardize(y_train)

plt.figure(figsize=(5,4))
plt.title('Training data')
plt.scatter(x_train[:,0], x_train[:,1], c=y_train, vmin=vmin, vmax=vmax)
plt.colorbar()
plt.xlim(0, 1); plt.ylim(0, 1)
plt.show()


## Model

With the data prepared, we are ready to define our model. Here we will use the GPyTorch package to define a GPR model. GPyTorch allows us to define the model in a modular way with a mean function, covariance function and likelihood with a noise model.

After defining the model, we can inspect all its submodules and parameters. 

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood)

print(model)

print()
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.item()}')

print()
for constraint_name, constraint in model.named_constraints():
    print(f'Constraint name: {constraint_name:55} constraint = {constraint}')

print()
print(f"noise:        {model.likelihood.noise_covar.noise.item()}")
print(f"mean:         {model.mean_module.constant.item()}")
print(f"output scale: {model.covar_module.outputscale.item()}")
print(f"length scale: {model.covar_module.base_kernel.lengthscale.item()}")


## Training

Now we can fit the model to the training data. With GPyTorch we can use gradient-based optimization very similar to how you train a neural network model using [PyTorch](https://pytorch.org/). We set our model in training mode, define an optimizer (we use ADAM in this case), an objective (loss) function and then define a training loop. 

After the training is complete, we should see the loss (error) on the training data going down. 

In [ ]:
training_iter = 100

# Initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(x_train, y_train, likelihood)

# Set model and likelihood into training mode
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

losses = []
for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(x_train)
    # Compute loss and backprop gradients
    loss = -mll(output, y_train)
    loss.backward()
    losses.append(loss.item())
    optimizer.step()
    if i == 0 or (i+1) % 10 == 0:
        print('Iter %d/%d - Loss: %.3f    variance: %.3f   lengthscale: %.3f   noise: %.3f' % (
            i + 1, training_iter, loss.item(),
            model.covar_module.outputscale.item(),
            model.covar_module.base_kernel.lengthscale.item(),
            model.likelihood.noise.item()
        ))

print()
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.item()}')

print()
print(f"noise:        {model.likelihood.noise_covar.noise.item()}")
print(f"mean:         {model.mean_module.constant.item()}")
print(f"output scale: {model.covar_module.outputscale.item()}")
print(f"length scale: {model.covar_module.base_kernel.lengthscale.item()}")

plt.figure(figsize=(6,3))
plt.title("Training loss")
plt.plot(losses)
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.show()


## Evaluation

Now that we have a trained model, we can evaluate how well it performs. 

For example, we can plot the true values versus the predicted values in what is called a parity plot. If the model fits the data, we expect the predictions to correspond to the true values. 

We call also compute error metrics such as the mean squared error](https://en.wikipedia.org/wiki/Mean_squared_error) (MSE) and [coefficient of determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) ($R^2$) to evaluate the predictions. If we have achieved a good fit, we should see a low error and an $R^2$ close to 1.0.

Finally, we expect the model to peform very well on the training dataset, since that is what we used to fit the model. To be useful, however, the model should also perform well on some unseen data. Here we will use the grid data we defined in the beginning as a test dataset, as this data covers the entire input space.

Note: In a real setting, you may not have access to a complete test dataset as we have here. Instead you can split your dataset into training and validation/test dataset. 

In [ ]:
def mse(y_true, y_pred):
    """Compute mean squared error."""
    return torch.mean((y_true - y_pred)**2)

def r2(y_true, y_pred):
    """Compute coefficient of determination."""
    ssr = torch.sum((y_true - y_pred)**2)
    sst = torch.sum((y_true - torch.mean(y_true))**2)
    return 1 - (ssr / sst)

# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

with torch.no_grad(), gpytorch.settings.fast_pred_var():
    f_pred = model(x_train)  # Model posterior distribution
    y_pred = likelihood(f_pred)  # Posterior predictive distribution

    y_pred_mean = y_pred.mean

    lim = (-3.5, 3.5)
    plt.figure(figsize=(3,3))
    plt.title("Training dataset")
    plt.plot(lim, lim, color="k", linewidth=1)
    plt.scatter(y_train, y_pred_mean)
    plt.xlim(lim); plt.ylim(lim)
    plt.xlabel("y_train"); plt.ylabel("y_pred_mean")
    plt.grid()
    plt.show()

    print("Training dataset")
    print(f"MSE:  {mse(y_train, y_pred_mean):7.4f}")
    print(f"R2:   {r2(y_train, y_pred_mean):7.4f}")

    f_pred = model(x_grid)  # Model posterior distribution
    y_pred = likelihood(f_pred)  # Posterior predictive distribution

    y_pred_mean = y_pred.mean
    y_pred_var = y_pred.variance

    lim = (-3.5, 3.5)
    plt.figure(figsize=(3,3))
    plt.title("Test dataset")
    plt.plot(lim, lim, color="k", linewidth=1)
    plt.scatter(y_grid, y_pred_mean, marker=".", alpha=0.1)
    plt.xlim(lim); plt.ylim(lim)
    plt.xlabel("y_grid"); plt.ylabel("y_pred_mean")
    plt.grid()
    plt.show()

    print("Test dataset")
    print(f"MSE:  {mse(y_grid, y_pred_mean):7.4f}")
    print(f"R2:   {r2(y_grid, y_pred_mean):7.4f}")


## Visualize predictions

Finally, we can visualize the model predictions and uncertainty to see what it has learned. If we have learned a good model, the predictions should resemble the true function. You should also see low uncertainty where we have observed some training data. 

Do you think the model can be improved any further? How?

In [ ]:
# plot predictions on grid

plt.figure(figsize=(5,4))
plt.title("Schwefel function")
plt.contourf(x0.numpy(), x1.numpy(), y_grid.reshape(n_grid, n_grid).numpy(), vmin=vmin, vmax=vmax, levels=levels)
plt.colorbar()
plt.show()

plt.figure(figsize=(5,4))
plt.title("Prediction mean")
plt.contourf(x0.numpy(), x1.numpy(), y_pred_mean.reshape(n_grid, n_grid).numpy(), vmin=vmin, vmax=vmax, levels=levels)
plt.scatter(x_train[:,0], x_train[:,1], c=y_train, vmin=vmin, vmax=vmax)  # plot training data
plt.colorbar()
plt.show()

plt.figure(figsize=(5,4))
plt.title("Prediction uncertainty")
plt.contourf(x0.numpy(), x1.numpy(), torch.sqrt(y_pred_var).reshape(n_grid, n_grid).numpy(), levels=levels)
plt.scatter(x_train[:,0], x_train[:,1], c="black")  # plot training data
plt.colorbar()
plt.show()


## Additional exercises

* Try to change the size of the training set.
* Try to change the noise level on the training set. 
* Try to change the input range to create a more complicated function.
* Try to change the kernel of the model.